In [5]:
import pandas as pd
from data_disaggregation import VT_NumericExt

In [2]:
ds = pd.Series([10]).rename_axis(['year'])
ds_map = pd.Series({(0, 0): 1, (0, 1): 3}).rename_axis(['year',"month"])


In [6]:
VT_NumericExt.disagg(ds, ds_map)

{0: 2.5, 1: 7.5}